## 导入依赖

In [15]:
import math
import numpy as np
import yaml

import sys
sys.path.append('../arm_5dof')
sys.path.append('../cv_color_block')

from cv_worksapce import CvWorkspace

## 载入示教数据

In [16]:
ws_9p_at_arm = None
with open("config/ws_9p_at_arm.yaml", "r") as f:
    ws_9p_at_arm = yaml.load(f,  Loader=yaml.FullLoader)

ws_9p_at_arm

{0: [-16.352997660133735, -6.607011818101981, 20.02660769873281],
 1: [-11.064548762701508, 0.09655848261909236, 24.095147596378297],
 2: [-16.231154170076213, 7.534217925167062, 19.771658278616258],
 3: [3.0624262238360873, 1.8111053248044202, 25.62745077906547],
 4: [11.738097261284942, -0.0, 22.539323606098574],
 5: [4.366432934288944, -2.6967511016051002, 25.176701374546155],
 6: [10.598491613600698, 10.709998794700551, 20.053839200332657],
 7: [18.032233496897817, 0.09441705329556965, 15.850156852473424],
 8: [6.920117435700747, -7.191021435962427, 23.19209821775296]}

## 计算工作台9点在工作台坐标系下的坐标

In [17]:
workspace = CvWorkspace(yaml_path='../cv_color_block/config/workspace.yaml')

In [18]:
# 注: 下面的计算单位都是cm
ws_w =  (workspace.ws_pp_tf['ws_img_width'] /  workspace.ws_pp_tf['ws_img_scale_factor'])
ws_h =  (workspace.ws_pp_tf['ws_img_height'] /  workspace.ws_pp_tf['ws_img_scale_factor'])

ws_9p_at_ws = {}
ws_9p_at_ws[0] = [ws_h/2, ws_w/2, 0]
ws_9p_at_ws[1] = [ws_h/2, 0, 0]
ws_9p_at_ws[2] = [ws_h/2, -ws_w/2, 0]
ws_9p_at_ws[3] = [0, ws_w/2, 0]
ws_9p_at_ws[4] = [0, 0, 0]
ws_9p_at_ws[5] = [0, -ws_w/2, 0]
ws_9p_at_ws[6] = [-ws_h/2, ws_w/2, 0]
ws_9p_at_ws[7] = [-ws_h/2, 0, 0]
ws_9p_at_ws[8] = [-ws_h/2, -ws_w/2, 0]

ws_9p_at_ws

{0: [0.063, 0.095, 0],
 1: [0.063, 0, 0],
 2: [0.063, -0.095, 0],
 3: [0, 0.095, 0],
 4: [0, 0, 0],
 5: [0, -0.095, 0],
 6: [-0.063, 0.095, 0],
 7: [-0.063, 0, 0],
 8: [-0.063, -0.095, 0]}

## 将点集转换为ndarray格式

In [19]:
ws_9p_at_arm_array = []
ws_9p_at_ws_array = []
for i in range(9):
    ws_9p_at_arm_array.append(ws_9p_at_arm[i])
    ws_9p_at_ws_array.append(ws_9p_at_ws[i])
ws_9p_at_arm_array = np.float32(ws_9p_at_arm_array)
ws_9p_at_ws_array = np.float32(ws_9p_at_ws_array)

## ICP算法-得到变换矩阵

In [20]:
# 分别计算两组点集的质心
ws_9p_at_arm_mean = np.mean(ws_9p_at_arm_array, axis=0)
ws_9p_at_ws_mean = np.mean(ws_9p_at_ws_array, axis=0)
print("ws_9p_at_arm_mean: {}".format(ws_9p_at_arm_mean))
print("ws_9p_at_ws_mean: {}".format(ws_9p_at_ws_mean))

ws_9p_at_arm_mean: [ 1.2299005   0.41683477 21.814775  ]
ws_9p_at_ws_mean: [1.6556846e-09 0.0000000e+00 0.0000000e+00]


In [21]:
# 点集去除质心
ws_9p_at_arm_center = ws_9p_at_arm_array - ws_9p_at_arm_mean
ws_9p_at_ws_center = ws_9p_at_ws_array - ws_9p_at_ws_mean

In [22]:
# 构造W矩阵
W = np.zeros((3, 3), dtype="float32")
for i in range(9):
    v1 = ws_9p_at_arm_center[i].reshape(-1, 1)
    v2 = ws_9p_at_ws_center[i].reshape(1, -1)
    W += v1.dot(v2)

In [23]:
# SVD分解
U,Sigma,Vt = np.linalg.svd(W)

In [24]:
# 求解旋转矩阵
R = U.dot(Vt)
R

array([[-0.9987682 , -0.00419024,  0.04944267],
       [ 0.00930105,  0.96295655,  0.26949608],
       [ 0.0487404 , -0.269624  ,  0.9617314 ]], dtype=float32)

In [25]:
ws_9p_at_arm_center

array([[-17.582897  ,  -7.0238466 ,  -1.788168  ],
       [-12.294449  ,  -0.3202763 ,   2.2803726 ],
       [-17.461054  ,   7.117383  ,  -2.0431175 ],
       [  1.8325258 ,   1.3942707 ,   3.8126755 ],
       [ 10.508197  ,  -0.41683477,   0.72454834],
       [  3.1365328 ,  -3.113586  ,   3.3619251 ],
       [  9.368591  ,  10.293164  ,  -1.7609367 ],
       [ 16.802334  ,  -0.32241774,  -5.9646187 ],
       [  5.690217  ,  -7.6078563 ,   1.3773232 ]], dtype=float32)

In [26]:
# 求解平移向量
t = ws_9p_at_arm_mean.reshape(-1,1) - R.dot(ws_9p_at_ws_mean.reshape(-1, 1))
t

array([[ 1.2299005 ],
       [ 0.41683477],
       [21.814775  ]], dtype=float32)

In [27]:
# 构造变换矩阵
# 工作台坐标系在Arm坐标系下的位姿描述
T_arm2ws = np.eye(4)
T_arm2ws[:3,:3] = R
T_arm2ws[:3, 3] = t.reshape(-1)

## 数据保存

In [28]:
np.savetxt("config/T_arm2ws.csv", T_arm2ws, delimiter=",")